In [1]:
#testing the image classifier with images from Reddit in a BLM context

In [ ]:
#installing all necessary libraries
#pip install opencv-python
#pip install tensorflow

In [1]:
#loading all necessary libraries 

#general libraries
import os
import glob
import pandas as pd
import random

#math libraries
import numpy as np

#visualization libraries 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

#image operations
import cv2 
from PIL import Image  

#deep learning libraries
import tensorflow as tf ##deep learning model 

#data format libraries
import csv 
import pickle

#nlp libraries 
import re


In [1]:
#check and change working directory
os.getcwd()
os.chdir("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/BLM_Images_Tumbnails")
os.getcwd()

In [2]:
#importing all pictures from folder and checking size and type for jpg
for image_path in glob.glob("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/BLM_Images_Tumbnails/*.jpg"):
    image = mpimg.imread(image_path)
    print (image.shape)
    print (image.dtype)
    plt.imshow(image)
    plt.show()
    print(os.path.basename(image_path))
    break

In [3]:
#checking all shapes of pictures jpg
for image_path in glob.glob("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Images_BLM_Dataset_direct_1/*.jpg"):
    try:
        image = mpimg.imread(image_path)
        print(image.shape)
    except Exception as e:
        print('Error, could not print picture', image_path)

In [4]:
#check for gray images because those are broken images which will not be converted to numpy array later on
gray_data = [] #empty array

for img in glob.glob("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/BLM_Images_Tumbnails/*.jpg"):
    try:
        image = mpimg.imread(img)#read image
        image_label = os.path.basename(img) #rename label
        if(len(image.shape)<3): #if smaller than 3 then it will be gray
            gray_data.append([image_label]) #append array
        else:
            print ('others')
    except Exception as e:
        pass

gray_data

In [7]:
#save gray data as list
with open("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Images_BLM_Dataset_direct_1/gray_data.txt", 'w') as output:
    for row in gray_data:
        output.write(str(row) + '\n')


In [9]:
#remove unnecessary items in txt file 
with open('/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Images_BLM_Dataset_direct_1/gray_data.txt', 'r') as my_file:
    text = my_file.read()
    text = text.replace("[", "")
    text = text.replace("]", "")
    text= text.replace("'", '')

#save the updates back into a cleaned up file
with open('/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Images_BLM_Dataset_direct_1/gray_data.txt', 'w') as my_file:
    my_file.write(text)

In [10]:
#delete images which are gray
path = "/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Images_BLM_Dataset_direct_1"
with open('gray_data.txt', "r") as list_file:
    _list = list_file.read().splitlines()
    [os.remove(os.path.join(path,f)) for f in _list]

In [5]:
#create own dataset with resized picutres
owntest_data = [] #empty array
IMG_SIZE = 255 #image size

for img in glob.glob("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/BLM_Images_Tumbnails/*.jpg"):
        try:
            image = mpimg.imread(img)
            image = image[:, :, :3] #dropping 4th dimension(RGB-4)
            new_array = cv2.resize(image, (IMG_SIZE, IMG_SIZE)) #resize
            print(new_array.shape) #check array size just to be sure
            image_label = os.path.basename(img) #rename label
            owntest_data.append([new_array, image_label]) #append array
        except Exception as e:
            print("Image is broken, move on", image_label)

#look at new dataset
owntest_data


In [6]:
#check length of array
print(len(owntest_data))

In [7]:
#check shape of array
print(new_array.shape)

In [8]:
#shuffle data otherwise it will not classify 
random.shuffle(owntest_data)

In [9]:
#have a look if data is really shuffled
for sample in owntest_data[:10]:
    print(sample[1])

In [9]:
#create lists for storing the data as np arrays
X = [] #picture array
y = [] #picture name array

for new_array, image_label in owntest_data:
    X.append(new_array)
    y.append(image_label)

In [10]:
#convert into numpy array
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [10]:
#look at the pictures
print(X)

In [11]:
#have a look at the labels 
print(y)

In [18]:
#save the dataset and labels 
pickle_out = open("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Images_BLM_Dataset_direct_1/X_BLMjpg.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Images_BLM_Dataset_direct_1/y_BLMjpg.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [19]:
#check if dataset loads
pickle_in = open("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Images_BLM_Dataset_direct_1/X_BLMjpg.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in = open("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Images_BLM_Dataset_direct_1/y_BLMjpg.pickle", "rb")
y = pickle.load(pickle_in)

In [12]:
#look at data
print(X.shape)

In [13]:
#look at picture labels
print(y)

In [11]:
#define categories where 0 = non-violent and 1 =violent and load model 
CATEGORIES = ["non-violent", "violent"]
model = tf.keras.models.load_model("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Python/CNN12.model")

In [14]:
#test pictures in model and looping through array 
#create lists to save data 
Post_ID = []
Violence_Classification = []

i = 0
while i < len(X):
    try:
        prediction = model.predict(X)
        for i in range(len(X)):
            print("Picture=%s, Prediction=%s" % (y[i], prediction[i]))
            Post_ID.append(y[0+i])
            Violence_Classification.append(prediction[0+i])
            #print(y[0+i], prediction[0+i])
            i+=1
    except Exception as e:
        print('Error', y[0+i]) 
        i+=1

In [15]:
#look at Post ID
Post_ID

In [16]:
#replace the extension .jpg
j=0
for i in Post_ID:
    Post_ID[j] =i.replace(".jpg", "")
    j+=1
print (Post_ID)

In [17]:
#convert numpay list to normal list in order to get fitting format 
my_dt = pd.DataFrame(Violence_Classification)
Violence = [i[0] for i in my_dt.values.tolist()]

print(type(Violence_Classification),type(my_dt),type(Violence))

In [18]:
#look at new list 
Violence

In [19]:
#rounding values to only get 1 or 0
Violence_rounded = [round(num) for num in Violence]
Violence_rounded

In [18]:
#save data to df and then to csv
df = pd.DataFrame(data={"Post ID": Post_ID, "Violence Classification": Violence_rounded})
df.to_csv("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/BLM_Images_Tumbnails/Violence.csv", sep=';',index=False)